# wordcount

In [ ]:
text = sc.textFile('in/hello.txt')
text.flatMap(lambda line: line.split()) \
        .map(lambda word: (word,1)) \
        .reduceByKey(lambda x,y: x + y) \
        .sortBy(lambda t: -t[1]) \
        .collect()

# middle num

In [ ]:
from operator import add
rdd = sc.textFile('in/num.txt')
# num of each num
mappeddata = rdd.map(lambda x: (int(x),1)) \
                .reduceByKey(add) \
#                .sortByKey()
count = mappeddata.reduce(lambda x,y: (0,x[1] + y[1]))[1]
mid = count // 2
print('count is:', count, '; mid is:', mid)
class FindNth:
    def __init__(self, n):
        self.n = n
        self.num = 0
    def __call__(self, x):
        self.num += x[1]
        if self.num < self.n:
            return False
        elif self.num == self.n:
            return True
        else:
            raise StopIteration
rs = mappeddata.filter(FindNth(mid)) \
                .collect() \
                [0][0]
print('the mid num is:', rs)

# inverted index

In [ ]:
rdd = sc.textFile('in/doc.txt')
def mapId(line):
    values = line.split()
    return [(values[i],values[0]) for i in range(1,len(values))]
rdd.flatMap(mapId) \
    .reduceByKey(lambda x,y: x + ',' + y) \
    .sortByKey() \
    .map(lambda t: t[0] + ' ' + t[1]) \
    .saveAsTextFile('out/index')

In [ ]:
rdd = sc.parallelize([1,2,3,4,5,1,2,3,4,5,9], 2)
rdd.reduce(lambda x,y: x^y)

# Join

In [ ]:
from operator import add
person = sc.textFile('in/person').map(lambda line: line.split())
action = sc.textFile('in/action').map(lambda line: line.split())

maxkey = action.sample(False, .5, 9) \
                .map(lambda x: (x[0],1)) \
                .reduceByKey(add) \
                .max(lambda x: x[1])[0]
print('max key is:', maxkey)

maxpart = action.filter(lambda p: p[0] == maxkey)
other = action.filter(lambda p: p[0] != maxkey)

person.join(maxpart) \
        .union(person.join(other)) \
        .collect()
#person.map(lambda line: line.split()) \
#        .partitionBy(2, lambda k: 0 if k == maxkey else 1) \
#        .glom().collect()